In [3]:
using SDDP
import Ipopt
import PowerModels
import Test

[ Info: Precompiling PowerModels [c36e90e8-916a-50a6-bd94-075b64ef4655] 


In [4]:
function build_model(model_type)
    filename = joinpath(@__DIR__, "pglib_opf_case5_pjm.m")
    data = PowerModels.parse_file(filename)
    return SDDP.PolicyGraph(
        SDDP.UnicyclicGraph(0.95);
        sense = :Min,
        lower_bound = 0.0,
        optimizer = Ipopt.Optimizer,
    ) do sp, t
        power_model = PowerModels.instantiate_model(
            data,
            model_type,
            PowerModels.build_opf;
            jump_model = sp,
        )
        # Now add hydro power models. Assume that generator 5 is hydro, and the
        # rest are thermal.
        pg = power_model.var[:it][:pm][:nw][0][:pg][5]
        sp[:pg] = pg
        @variable(sp, x >= 0, SDDP.State, initial_value = 10.0)
        @variable(sp, deficit >= 0)
        @constraint(sp, balance, x.out == x.in - pg + deficit)
        @stageobjective(sp, objective_function(sp) + 1e6 * deficit)
        SDDP.parameterize(sp, [0, 2, 5]) do ω
            return SDDP.set_normalized_rhs(balance, ω)
        end
        return
    end
end

build_model (generic function with 1 method)

In [5]:
convex = build_model(PowerModels.DCPPowerModel)
SDDP.train(convex; iteration_limit = 10)

[info | PowerModels]: extending matpower format with data: areas 1x3
[info | PowerModels]: removing 1 cost terms from generator 4: [4000.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 1: [1400.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 5: [1000.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 2: [1500.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 3: [3000.0, 0.0]
-------------------------------------------------------------------
         SDDP.jl (c) Oscar Dowson and contributors, 2017-25
-------------------------------------------------------------------
problem
  nodes           : 1
  state variables : 1
  scenarios       : Inf
  existing cuts   : false
options
  solver          : serial mode
  risk measure    : SDDP.Expectation()
  sampling scheme : SDDP.InSampleMonteCarlo
subproblem structure
  VariableRef                             : [20, 20]
  AffExpr in MOI.EqualTo{Float64}         : [13, 13]
  A

In [6]:
SDDP.write_cuts_to_file(convex, "convex.cuts.json")
non_convex = build_model(PowerModels.ACPPowerModel)
SDDP.read_cuts_from_file(non_convex, "convex.cuts.json")

[info | PowerModels]: extending matpower format with data: areas 1x3
[info | PowerModels]: removing 1 cost terms from generator 4: [4000.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 1: [1400.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 5: [1000.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 2: [1500.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 3: [3000.0, 0.0]


In [7]:
result = SDDP.simulate(non_convex, 1)

1-element Vector{Vector{Dict{Symbol, Any}}}:
 [Dict(:bellman_term => 366369.0491268241, :noise_term => 0, :node_index => 1, :stage_objective => 21433.37406558235, :objective_state => nothing, :belief => Dict(1 => 1.0)), Dict(:bellman_term => 367827.88953612855, :noise_term => 2, :node_index => 1, :stage_objective => 21433.374065582408, :objective_state => nothing, :belief => Dict(1 => 1.0)), Dict(:bellman_term => 373320.09572670073, :noise_term => 0, :node_index => 1, :stage_objective => 21433.37406558299, :objective_state => nothing, :belief => Dict(1 => 1.0)), Dict(:bellman_term => 368728.8874641027, :noise_term => 5, :node_index => 1, :stage_objective => 21433.374065582466, :objective_state => nothing, :belief => Dict(1 => 1.0)), Dict(:bellman_term => 374221.09365467454, :noise_term => 0, :node_index => 1, :stage_objective => 21433.374065583223, :objective_state => nothing, :belief => Dict(1 => 1.0)), Dict(:bellman_term => 377708.2659637787, :noise_term => 0, :node_index => 1, :stag

In [8]:
non_convex = build_model(PowerModels.ACPPowerModel)
SDDP.train(non_convex; iteration_limit = 10)
result = SDDP.simulate(non_convex, 1)

[info | PowerModels]: extending matpower format with data: areas 1x3
[info | PowerModels]: removing 1 cost terms from generator 4: [4000.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 1: [1400.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 5: [1000.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 2: [1500.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 3: [3000.0, 0.0]
-------------------------------------------------------------------
         SDDP.jl (c) Oscar Dowson and contributors, 2017-25
-------------------------------------------------------------------
problem
  nodes           : 1
  state variables : 1
  scenarios       : Inf
  existing cuts   : false
options
  solver          : serial mode
  risk measure    : SDDP.Expectation()
  sampling scheme : SDDP.InSampleMonteCarlo
subproblem structure
  VariableRef                             : [48, 48]
  AffExpr in MOI.EqualTo{Float64}         : [12, 12]
  A

1-element Vector{Vector{Dict{Symbol, Any}}}:
 [Dict(:bellman_term => 412455.7091289238, :noise_term => 0, :node_index => 1, :stage_objective => 17598.687288058165, :objective_state => nothing, :belief => Dict(1 => 1.0)), Dict(:bellman_term => 411855.54849427275, :noise_term => 5, :node_index => 1, :stage_objective => 17598.687287409266, :objective_state => nothing, :belief => Dict(1 => 1.0)), Dict(:bellman_term => 416686.55843310436, :noise_term => 2, :node_index => 1, :stage_objective => 17598.687305606843, :objective_state => nothing, :belief => Dict(1 => 1.0)), Dict(:bellman_term => 419547.0716782107, :noise_term => 0, :node_index => 1, :stage_objective => 25327.453867795644, :objective_state => nothing, :belief => Dict(1 => 1.0)), Dict(:bellman_term => 417702.52959825343, :noise_term => 5, :node_index => 1, :stage_objective => 18618.385455716925, :objective_state => nothing, :belief => Dict(1 => 1.0))]

In [9]:
convex = build_model(PowerModels.DCPPowerModel)

[info | PowerModels]: extending matpower format with data: areas 1x3
[info | PowerModels]: removing 1 cost terms from generator 4: [4000.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 1: [1400.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 5: [1000.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 2: [1500.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 3: [3000.0, 0.0]


A policy graph with 1 nodes.
 Node indices: 1


In [10]:
non_convex = build_model(PowerModels.ACPPowerModel)

[info | PowerModels]: extending matpower format with data: areas 1x3
[info | PowerModels]: removing 1 cost terms from generator 4: [4000.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 1: [1400.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 5: [1000.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 2: [1500.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 3: [3000.0, 0.0]


A policy graph with 1 nodes.
 Node indices: 1


In [11]:
SDDP.train(
    convex;
    forward_pass = SDDP.AlternativeForwardPass(non_convex),
    post_iteration_callback = SDDP.AlternativePostIterationCallback(non_convex),
    iteration_limit = 10,
)

-------------------------------------------------------------------
         SDDP.jl (c) Oscar Dowson and contributors, 2017-25
-------------------------------------------------------------------
problem
  nodes           : 1
  state variables : 1
  scenarios       : Inf
  existing cuts   : false
options
  solver          : serial mode
  risk measure    : SDDP.Expectation()
  sampling scheme : SDDP.InSampleMonteCarlo
subproblem structure
  VariableRef                             : [20, 20]
  AffExpr in MOI.EqualTo{Float64}         : [13, 13]
  AffExpr in MOI.Interval{Float64}        : [6, 6]
  VariableRef in MOI.GreaterThan{Float64} : [14, 14]
  VariableRef in MOI.LessThan{Float64}    : [11, 11]
numerical stability report
  matrix range     [1e+00, 2e+02]
  objective range  [1e+00, 1e+06]
  bounds range     [4e-01, 6e+00]
  rhs range        [5e-01, 5e+00]
-------------------------------------------------------------------
 iteration    simulation      bound        time (s)     solves  